In [12]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_predict, cross_validate, GridSearchCV

from sklearn.preprocessing import RobustScaler, PowerTransformer

import itertools

from sklearn.metrics.pairwise import paired_distances

import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from sklearn.compose import ColumnTransformer
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

from neo4j import GraphDatabase

In [26]:
df = pd.read_excel('/home/guest/Загрузки/Basheer_demo.xlsx')

In [27]:
[x.replace('l0_', '') for x in df.columns if 'l0_' in x][:10]
df[[x for x in df.columns if 'l0_' in x][:10]]

,l0_42_03257085331836,l0_43_04741898699762,l0_44_01450529149254,l0_44_0489926172728,l0_44_99144370755021,l0_45_99197266970869,l0_47_04077398412186,l0_48_045099177099885,l0_49_00477743760921,l0_49_99501251131184
0,0.067512,0.628985,0.079979,0.065705,3.712502,0.065790,0.103783,0.005272,0.046767,0.007219
1,0.052837,0.293217,0.041497,0.027165,3.342605,0.056691,0.089783,0.002514,0.008929,0.004855
2,0.034711,0.289125,0.039634,0.023119,3.528768,0.058900,0.210027,0.006351,0.027325,0.005452
3,0.065292,0.965013,0.055779,0.059098,2.963412,0.052547,0.143320,0.004360,0.017409,0.006269
4,0.080594,3.142379,0.052536,0.122263,2.635041,0.043910,0.100054,0.002831,0.004470,0.004573
...,...,...,...,...,...,...,...,...,...,...
75,0.070222,1.218192,0.060510,0.058470,3.800003,0.064128,0.162522,0.004768,0.066721,0.009513
76,0.083748,2.472044,0.110516,0.126304,2.979950,0.058456,8.012591,0.178858,0.030891,0.008964
77,0.059168,2.266683,0.108345,0.127781,1.556364,0.071685,0.186592,0.005847,0.043078,0.005111
78,0.288774,1.230780,0.062873,0.050527,1.983028,0.051258,0.217254,0.006029,0.002695,0.002760


In [28]:
vocs_names = [x.replace('l0_', '') for x in df.columns if 'l0_' in x][:10]

df_vocs = df[[x for x in df.columns if 'l0_' in x][:10]]
df_vocs.columns = vocs_names

unique_probe_id = ['ProbeId_' + str(x) for x in range(df.shape[0])]
df_vocs['PatientID'] = df['Cardio_Qvark_ID']
df_vocs['ProbeId'] = ['ProbeId_' + str(x) for x in range(df.shape[0])]
df_vocs['DateTime'] = [datetime.now() for x in range(df.shape[0])]

y = (df['Myocardial_perfusion_defect_after_stress_ATP'] == 'Yes').astype(float)
df_vocs['target'] =y.replace([0, 1], ['Healthy', 'ИБС'])

X_train, _, y_train, _ = train_test_split(df_vocs.drop('target', axis = 'columns'), y, test_size=0.8, random_state=42)
X_train['target_short_name'] =y_train.replace([0, 1], ['Healthy', 'ИБС'])
X_train['target_icd_10'] =y_train.replace([0, 1], ['Healthy', 'I20.8'])
X_train['target_icd_11'] =y_train.replace([0, 1], ['Healthy', 'BA40.1'])

df_vocs = X_train

df1 = df_vocs

print(df_vocs.to_markdown())


|    |   42_03257085331836 |   43_04741898699762 |   44_01450529149254 |   44_0489926172728 |   44_99144370755021 |   45_99197266970869 |   47_04077398412186 |   48_045099177099885 |   49_00477743760921 |   49_99501251131184 |   PatientID | ProbeId    | DateTime                   | target_short_name   | target_icd_10   | target_icd_11   |
|---:|--------------------:|--------------------:|--------------------:|-------------------:|--------------------:|--------------------:|--------------------:|---------------------:|--------------------:|--------------------:|------------:|:-----------|:---------------------------|:--------------------|:----------------|:----------------|
| 59 |           0.0815782 |            0.894392 |           0.0659017 |          0.0497411 |             3.30478 |           0.0727878 |            0.195982 |           0.00624026 |          0.043772   |          0.0103969  |       19947 | ProbeId_59 | 2024-08-31 11:21:02.203422 | Healthy             | Healthy      

In [142]:
vocs_names = [x.replace('l0_', '') for x in df.columns if 'l0_' in x][4:10]

df_vocs = df[[x for x in df.columns if 'l0_' in x][4:10]]
df_vocs.columns = vocs_names

unique_probe_id = ['ProbeId_' + str(x) for x in range(df.shape[0])]
df_vocs['PatientID'] = df['Cardio_Qvark_ID']
df_vocs['ProbeId'] = ['ProbeId_' + str(x) for x in range(df.shape[0])]
df_vocs['DateTime'] = [datetime.now() for x in range(df.shape[0])]

y = (df['Myocardial_perfusion_defect_after_stress_ATP'] == 'Yes').astype(float)
df_vocs['target'] =y.replace([0, 1], ['Healthy', 'ИБС'])

X_train, _, y_train, _ = train_test_split(df_vocs.drop('target', axis = 'columns'), y, test_size=.9, random_state=58)
X_train['target_short_name'] =y_train.replace([0, 1], ['Healthy', 'ИБС'])
X_train['target_icd_10'] =y_train.replace([0, 1], ['Healthy', 'I20.8'])
X_train['target_icd_11'] =y_train.replace([0, 1], ['Healthy', 'BA40.1'])

df_vocs = X_train

df2 = df_vocs

print(df_vocs.to_markdown())

|    |   44_99144370755021 |   45_99197266970869 |   47_04077398412186 |   48_045099177099885 |   49_00477743760921 |   49_99501251131184 |   PatientID | ProbeId    | DateTime                   | target_short_name   | target_icd_10   | target_icd_11   |
|---:|--------------------:|--------------------:|--------------------:|---------------------:|--------------------:|--------------------:|------------:|:-----------|:---------------------------|:--------------------|:----------------|:----------------|
| 76 |             2.97995 |           0.0584557 |            8.01259  |           0.178858   |           0.0308905 |          0.00896429 |       22015 | ProbeId_76 | 2024-08-30 19:37:52.812337 | Healthy             | Healthy         | Healthy         |
| 79 |             1.92739 |           0.0499004 |            0.192831 |           0.00517879 |           0.0379916 |          0.00480427 |       23843 | ProbeId_79 | 2024-08-30 19:37:52.812339 | Healthy             | Healthy         | He

In [23]:
import pandas as pd
from neo4j import GraphDatabase

# Define the Neo4j connection parameters
NEO4J_URI = "neo4j://localhost:7687"  # Change if needed
NEO4J_USER = ""  # Your username
NEO4J_PASSWORD = ""  # Your password

# Load the data into a DataFrame
data = {
    "526_03257085331836": [0.0347108, 0.936501, 0.0703742, 0.0917151, 0.741693, 0.074873, 0.106864, 0.171386],
    "742_04741898699762": [0.289125, 1.91298, 2.49884, 1.27432, 0.964586, 0.811691, 3.14315, 3.38432],
    "44_01450529149254": [0.0396336, 0.0739828, 0.0789201, 0.0940473, 0.0550934, 0.0709599, 0.0991804, 0.0795997],
    "49_0489926172728": [0.0231194, 0.087051, 0.101689, 0.0762481, 0.0501778, 0.0548712, 0.139365, 0.137908],
    "96_99144370755021": [3.52877, 3.51884, 2.68527, 5.33148, 3.35456, 3.9698, 4.15313, 2.83617],
    "PersonID": [17558, 21569, 21094, 19907, 20195, 21387, 20007, 18380],
    "ProbeId": ["ProbeId_2", "ProbeId_23", "ProbeId_74", "ProbeId_20", "ProbeId_60", "ProbeId_71", "ProbeId_14", "ProbeId_51"],
    "DateTime": [
        "2024-08-30 23:45:47.877923", "2024-08-30 23:45:47.877936", "2024-08-30 23:45:47.877968",
        "2024-08-30 23:45:47.877934", "2024-08-30 23:45:47.877960", "2024-08-30 23:45:47.877967",
        "2024-08-30 23:45:47.877930", "2024-08-30 23:45:47.877954"
    ],
    "target_short_name": ["ИБС", "ИБС", "Healthy", "ИБС", "Healthy", "Healthy", "ИБС", "Healthy"],
    "target_icd_10": ["I20.8", "I20.8", "Healthy", "I20.8", "Healthy", "Healthy", "I20.8", "Healthy"],
    "target_icd_11": ["BA40.1", "BA40.1", "Healthy", "BA40.1", "Healthy", "Healthy", "BA40.1", "Healthy"]
}

df = pd.DataFrame(data)

# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def create_graph(tx, person_id, probe_id, date_time, breathing_type, normed, vocs, target):
    # Create Person node
    tx.run("MERGE (p:Person {PersonID: $person_id})", person_id=person_id)

    # Create Volatome node
    tx.run("MERGE (v:Volatome {ProbeId: $probe_id}) "
           "SET v.DateTime = $date_time, v.BreathingType = $breathing_type, v.Normed = $normed",
           probe_id=probe_id, date_time=date_time, breathing_type='Normal', normed='Raw')

    # Create VOC nodes and establish relationships
    for voc_name, voc_value in vocs.items():
        tx.run("MERGE (vo:VOC {Name: $voc_name}) "
               "SET vo.Value = $voc_value", voc_name=voc_name, voc_value=voc_value)
        tx.run("MATCH (v:Volatome {ProbeId: $probe_id}), (vo:VOC {Name: $voc_name}) "
               "MERGE (v)-[:CONTAINS]->(vo)", probe_id=probe_id, voc_name=voc_name)

    # Create Target node
    tx.run("MERGE (t:Target {ShortName: $short_name, ICD10: $icd10, ICD11: $icd11})", 
           short_name=target['short_name'], icd10=target['icd_10'], icd11=target['icd_11'])
    tx.run("MATCH (v:Volatome {ProbeId: $probe_id}), (t:Target {ShortName: $short_name, ICD10: $icd10, ICD11: $icd11}) "
           "MERGE (v)-[:ASSOCIATED_WITH]->(t)", 
           probe_id=probe_id, short_name=target['short_name'], icd10=target['icd_10'], icd11=target['icd_11'])

    # Establish relationship between Person and Volatome
    tx.run("MATCH (p:Person {PersonID: $person_id}), (v:Volatome {ProbeId: $probe_id}) "
           "MERGE (p)-[:BREATHES]->(v)", person_id=person_id, probe_id=probe_id)

# Import data into Neo4j
with driver.session() as session:
    for index, row in df.iterrows():
        vocs = {
            '526_03257085331836': row['526_03257085331836'],
            '742_04741898699762': row['742_04741898699762'],
            '44_01450529149254': row['44_01450529149254'],
            '49_0489926172728': row['49_0489926172728'],
            '96_99144370755021': row['96_99144370755021']
        }
        target = {
            'short_name': row['target_short_name'],
            'icd_10': row['target_icd_10'],
            'icd_11': row['target_icd_11']
        }
        session.write_transaction(create_graph, row['PersonID'], row['ProbeId'], row['DateTime'], 'Normal', 'Raw', vocs, target)

# Close the driver
driver.close()

In [42]:
import pandas as pd
from neo4j import GraphDatabase

# Define the Neo4j connection parameters
NEO4J_URI = "bolt://localhost:7687"  # Change if needed
NEO4J_USER = ""  # Your username
NEO4J_PASSWORD = ""  # Your password

# Load the data into a DataFrame
data = {
    "526.03257085331836": [0.0347108, 0.936501, 0.0703742, 0.0917151, 0.741693, 0.074873, 0.074873, 0.106864, 0.171386],
    "742.04741898699762": [0.289125, 1.91298, 2.49884, 1.27432, 0.964586, 0.811691, 0.811691, 3.14315, 3.38432],
    "44.01450529149254": [0.0396336, 0.0739828, 0.0789201, 0.0940473, 0.0550934, 0.0709599, 0.0709599, 0.0991804, 0.0795997],
    "49.0489926172728": [0.0231194, 0.087051, 0.101689, 0.0762481, 0.0501778, 0.0548712, 0.0548712, 0.139365, 0.137908],
    "96.99144370755021": [3.52877, 3.51884, 2.68527, 5.33148, 3.35456, 3.9698, 3.9698, 4.15313, 2.83617],
    "PersonID": [17558, 21569, 21094, 19907, 20195, 21387, 21387, 20007, 18380],
    "ProbeId": ["ProbeId_2", "ProbeId_23", "ProbeId_74", "ProbeId_20", "ProbeId_60", "ProbeId_71", "ProbeId_70", "ProbeId_14", "ProbeId_51"],
    "Breathing": ["Normal", "Normal", "FRC", "FRC", "Normal", "Normal", "Normal", "Normal", "Normal"],
    "Normed": ["Raw", "Raw", "Raw", "Raw", "Raw", "[2H]O[2H]", "Raw", "Raw", "Raw"],
    "DateTime": [
        "2024-08-30 23:45:47.877923", "2024-08-30 23:45:47.877936", "2024-08-30 23:45:47.877968",
        "2024-08-30 23:45:47.877934", "2024-08-30 23:45:47.877960", "2024-08-30 23:45:47.877967",
        "2024-08-30 23:47:52", "2024-08-30 23:45:47.877930", "2024-08-30 23:45:47.877954"
    ],
    "target_short_name": ["ИБС", "ИБС", "Healthy", "ИБС", "Healthy", "Healthy", "Healthy", "ИБС", "Healthy"],
    "target_icd_10": ["I20.8", "I20.8", "Healthy", "I20.8", "Healthy", "Healthy", "Healthy", "I20.8", "Healthy"],
    "target_icd_11": ["BA40.1", "BA40.1", "Healthy", "BA40.1", "Healthy", "Healthy", "Healthy", "BA40.1", "Healthy"]
}

df = pd.DataFrame(data)

# List of substances
substances = [f"Unknown{i}" for i in range(len(df.columns) - 6)]  # Create Unknown0, Unknown1, ...

# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def create_graph(tx, person_id, probe_id, date_time, breathing, normed, vocs, target):
    # Create Person node
    tx.run("MERGE (p:Person {PersonID: $person_id})", person_id=person_id)

    # Create Volatome node
    tx.run("MERGE (v:Volatome {ProbeId: $probe_id}) "
           "SET v.DateTime = $date_time, v.BreathingType = $breathing",
           probe_id=probe_id, date_time=date_time, breathing=breathing)

    # Create VOC nodes and establish relationships
    for i, (voc_name, voc_value) in enumerate(vocs.items()):
        substance_name = substances[i]  # Get corresponding substance name
        tx.run("MERGE (vo:VOC {Name: $voc_name}) "
               "SET vo.Value = $voc_value", voc_name=voc_name, voc_value=voc_value)
        tx.run("MATCH (v:Volatome {ProbeId: $probe_id}), (vo:VOC {Name: $voc_name}) "
               "MERGE (v)-[:CONTAINS]->(vo)", probe_id=probe_id, voc_name=voc_name)

        # Connect VOCs to Substance
        tx.run("MERGE (s:Substance {Name: $substance_name})", substance_name=substance_name)
        tx.run("MATCH (vo:VOC {Name: $voc_name}), (s:Substance {Name: $substance_name}) "
               "MERGE (vo)-[:IS_MASKED_BY]->(s)", voc_name=voc_name, substance_name=substance_name)

    # Create Target node
    tx.run("MERGE (t:Target {ShortName: $short_name, ICD10: $icd10, ICD11: $icd11})", 
           short_name=target['short_name'], icd10=target['icd_10'], icd11=target['icd_11'])
    tx.run("MATCH (v:Volatome {ProbeId: $probe_id}), (t:Target {ShortName: $short_name, ICD10: $icd10, ICD11: $icd11}) "
           "MERGE (v)-[:ASSOCIATED_WITH]->(t)", 
           probe_id=probe_id, short_name=target['short_name'], icd10=target['icd_10'], icd11=target['icd_11'])

    # Establish relationship between Person and Volatome
    tx.run("MATCH (p:Person {PersonID: $person_id}), (v:Volatome {ProbeId: $probe_id}) "
           "MERGE (p)-[:BREATHES]->(v)", person_id=person_id, probe_id=probe_id)

# Import data into Neo4j
with driver.session() as session:
    for index, row in df.iterrows():
        # Define VOCs as a dictionary
        vocs = {
            '526.03257085331836': row['526.03257085331836'],
            '742.04741898699762': row['742.04741898699762'],
            '44.01450529149254': row['44.01450529149254'],
            '49.0489926172728': row['49.0489926172728'],
            '96.99144370755021': row['96.99144370755021']
        }
        
        # Define target as a dictionary
        target = {
            'short_name': row['target_short_name'],
            'icd_10': row['target_icd_10'],
            'icd_11': row['target_icd_11']
        }
        
        # Call the function to create the graph
        session.write_transaction(create_graph, 
                                   row['PersonID'], 
                                   row['ProbeId'], 
                                   row['DateTime'], 
                                   row['Breathing'], 
                                   row['Normed'], 
                                   vocs, 
                                   target)

# Close the driver
# driver.close()

In [34]:
df.shape

(9, 13)

In [57]:
import pandas as pd
from neo4j import GraphDatabase

# # Define the Neo4j connection parameters
# NEO4J_URI = "bolt://localhost:7687"  # Change if needed
# NEO4J_USER = "neo4j"  # Your username
# NEO4J_PASSWORD = "password"  # Your password

# # Connect to Neo4j
# driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def fetch_volatomes_with_target(tx, target_short_name):
    query = """
    MATCH (t:Target {ShortName: $target_short_name})<-[:ASSOCIATED_WITH]-(v:Volatome)-[:CONTAINS]->(vo:VOC)
    RETURN v.ProbeId AS ProbeId, vo.Name AS VOC_Name, vo.Value AS VOC_Value
    """
    result = tx.run(query, target_short_name=target_short_name)
    
    data = []
    for record in result:
        data.append({
            'ProbeId': record['ProbeId'],
            record['VOC_Name']: record['VOC_Value']
        })
    
    return pd.DataFrame(data)

# Fetch data from Neo4j
with driver.session() as session:
    df = session.read_transaction(fetch_volatomes_with_target, 'ИБС')

# print(df.to_markdown())
df
# # Pivot the DataFrame to have VOCs as columns
# df_pivot = df.pivot(index='ProbeId', columns=[col for col in df.columns if col != 'ProbeId'], values=[col for col in df.columns if col != 'ProbeId']).reset_index()

# # Flatten the MultiIndex columns
# df_pivot.columns = ['_'.join(str(col)).strip() for col in df_pivot.columns.values]

# # Display the resulting DataFrame
# print(df_pivot)
# df_pivot
# # Close the driver
# # driver.close()

,ProbeId,96.99144370755021,44.01450529149254,49.0489926172728,742.04741898699762,526.03257085331836,526_03257085331836,96_99144370755021,742_04741898699762,49_0489926172728,44_01450529149254
0,ProbeId_14,2.83617,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ProbeId_14,NaN,0.0796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ProbeId_14,NaN,NaN,0.137908,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ProbeId_14,NaN,NaN,NaN,3.38432,NaN,NaN,NaN,NaN,NaN,NaN
4,ProbeId_14,NaN,NaN,NaN,NaN,0.171386,NaN,NaN,NaN,NaN,NaN
5,ProbeId_14,NaN,NaN,NaN,NaN,NaN,0.171386,NaN,NaN,NaN,NaN
6,ProbeId_14,NaN,NaN,NaN,NaN,NaN,NaN,2.83617,NaN,NaN,NaN
7,ProbeId_14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.38432,NaN,NaN
8,ProbeId_14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.137908,NaN
9,ProbeId_14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0796
